In [ ]:
# Uninstall old versions
!pip uninstall -y moviepy decorator

# Install latest MoviePy
%pip install moviepy

Found existing installation: moviepy 2.2.1
Uninstalling moviepy-2.2.1:
  Successfully uninstalled moviepy-2.2.1
Found existing installation: decorator 5.2.1
Uninstalling decorator-5.2.1:
  Successfully uninstalled decorator-5.2.1
  Using cached moviepy-2.2.1-py3-none-any.whl.metadata (6.9 kB)
  Using cached decorator-5.2.1-py3-none-any.whl.metadata (3.9 kB)
Using cached moviepy-2.2.1-py3-none-any.whl (129 kB)
Using cached decorator-5.2.1-py3-none-any.whl (9.2 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [moviepy]


In [2]:
# FILE_NAME= "audio_helper.py" -
## INSERT FOLLOWING CODE TO SEPERATE PYTHON FILE "audio_helper.py"

import os
from pathlib import Path
from moviepy import VideoFileClip
from datetime import datetime

def parse_timestamp(timestamp_str):
    """Convert MM:SS timestamp string to seconds."""
    try:
        parts = timestamp_str.split(':')
        if len(parts) != 2:
            raise ValueError("Timestamp must be in MM:SS format")

        minutes = int(parts[0])
        seconds = int(parts[1])

        if minutes < 0 or seconds < 0 or seconds >= 60:
            raise ValueError("Invalid time values")

        return minutes * 60 + seconds
    except Exception as e:
        raise ValueError(f"Invalid timestamp format '{timestamp_str}': {e}")

def extract_audio_from_video(video_path, start_time_str=None, end_time_str=None, output_filename=None):
    """
    Extract audio from video file. If no timestamps provided, extracts full audio.

    Args:
        video_path: Path to video file
        start_time_str: Optional start time in MM:SS format (None = start of video)
        end_time_str: Optional end time in MM:SS format (None = end of video)
        output_filename: Optional custom output filename (without extension)

    Returns:
        Path to saved audio file
    """
    # Create output directory
    output_dir = Path("/Users/libiv/code/VERA/data/raw/extracted_audio")
    output_dir.mkdir(exist_ok=True)

    # Load video first to get duration
    print(f"\n{'='*70}")
    print(f"Video Audio Extractor")
    print(f"{'='*70}")
    print(f"Video: {video_path}")
    print(f"Loading video...")

    video = VideoFileClip(video_path)
    print(f"Video duration: {video.duration:.2f} seconds")

    # Parse timestamps or use full video duration
    if start_time_str is None:
        start_seconds = 0
    else:
        start_seconds = parse_timestamp(start_time_str)

    if end_time_str is None:
        end_seconds = video.duration
    else:
        end_seconds = parse_timestamp(end_time_str)

    # Generate output filename
    if output_filename:
        output_path = output_dir / f"{output_filename}.mp3"
    else:
        video_basename = Path(video_path).stem
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        if start_time_str is None and end_time_str is None:
            output_path = output_dir / f"{video_basename}_full_{timestamp}.mp3"
        else:
            output_path = output_dir / f"{video_basename}_slice_{timestamp}.mp3"

    print(f"Start: {start_seconds:.2f}s")
    print(f"End: {end_seconds:.2f}s")
    print(f"{'='*70}\n")

    # Validate timestamps
    if start_seconds < 0 or end_seconds > video.duration:
        video.close()
        raise ValueError(f"Timestamps out of range. Video duration: {video.duration:.2f}s")

    if start_seconds >= end_seconds:
        video.close()
        raise ValueError("Start time must be before end time")

    # Extract audio
    print(f"Extracting audio from {start_seconds:.2f}s to {end_seconds:.2f}s...")

    # If extracting full audio, no need to subclip
    if start_seconds == 0 and end_seconds == video.duration:
        audio_clip = video.audio
    else:
        audio_clip = video.subclipped(start_seconds, end_seconds).audio

    # Save audio
    print(f"Saving audio to: {output_path}")
    audio_clip.write_audiofile(str(output_path), codec='mp3', bitrate='192k', logger=None)

    # Cleanup
    audio_clip.close()
    video.close()

    print(f"\n{'='*70}")
    print(f"SUCCESS! Audio saved to: {output_path}")
    print(f"{'='*70}\n")

    return str(output_path)

# ============= USAGE  =============
if __name__ == "__main__":
    video_path = "/Users/libiv/code/VERA/data/raw/videos/TED_young_speaker.mp4"

    # Example 1: Extract full audio (no timestamps)
    try:
        audio_file = extract_audio_from_video(video_path)
        print(f"Done! Your audio is ready at: {audio_file}")
    except Exception as e:
        print(f"Error: {e}")

    # Example 2: Extract with custom output name
    # try:
    #     audio_file = extract_audio_from_video(video_path, output_filename="my_full_audio")
    #     print(f"Done! Your audio is ready at: {audio_file}")
    # except Exception as e:
    #     print(f"Error: {e}")

    # Example 3: Extract specific time range
    # try:
    #     audio_file = extract_audio_from_video(video_path, "00:00", "00:15")
    #     print(f"Done! Your audio is ready at: {audio_file}")
    # except Exception as e:
    #     print(f"Error: {e}")



Video Audio Extractor
Video: /Users/libiv/code/VERA/data/raw/videos/TED_young_speaker.mp4
Loading video...
Video duration: 15.38 seconds
Start: 0.00s
End: 15.38s

Extracting audio from 0.00s to 15.38s...
Saving audio to: /Users/libiv/code/VERA/data/raw/extracted_audio/TED_young_speaker_full_20251203_182009.mp3

SUCCESS! Audio saved to: /Users/libiv/code/VERA/data/raw/extracted_audio/TED_young_speaker_full_20251203_182009.mp3

Done! Your audio is ready at: /Users/libiv/code/VERA/data/raw/extracted_audio/TED_young_speaker_full_20251203_182009.mp3


In [ ]:
# # FILE_NAME=  main_app.py  -
## INSERT FOLLOWING CODE TO main_app.py

# 1. Import the function from your file (audio_helper.py)
from audio_helper import extract_audio_from_video

# 2. Set up your variables
my_video = "/Users/libiv/code/VERA/data/raw/videos/TED_young_speaker.mp4"

print("Starting extraction from main app...")

# 3. Call the function!
# This will run the function without running the "if __name__" block from the other file
audio_path = extract_audio_from_video(my_video)

print(f"Great success! File is at: {audio_path}")


Starting extraction from main app...

Video Audio Extractor
Video: /Users/libiv/code/VERA/data/raw/videos/TED_young_speaker.mp4
Loading video...
Video duration: 15.38 seconds
Start: 0.00s
End: 15.38s

Extracting audio from 0.00s to 15.38s...
Saving audio to: /Users/libiv/code/VERA/data/raw/extracted_audio/TED_young_speaker_full_20251203_182015.mp3

SUCCESS! Audio saved to: /Users/libiv/code/VERA/data/raw/extracted_audio/TED_young_speaker_full_20251203_182015.mp3

Great success! File is at: /Users/libiv/code/VERA/data/raw/extracted_audio/TED_young_speaker_full_20251203_182015.mp3
